In [1]:
import pandas as pd
import numpy as np
import re
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import f1_score

In [3]:
data = pd.read_csv('/content/spam.csv',encoding='latin')

In [4]:
data

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [5]:
data.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'], axis=1, inplace=True)

In [6]:
encoder = LabelEncoder()

data['v1'] = encoder.fit_transform(data['v1'])
class_mappings = {index: label for index, label in enumerate(encoder.classes_)}
class_mappings

{0: 'ham', 1: 'spam'}

In [8]:
def processEmail(contents):
    ps = PorterStemmer()

    contents= contents.lower()
    contents = re.sub(r'<[^<>]+>', ' ', contents)
    contents = re.sub(r'[0-9]+', 'number', contents)
    contents = re.sub(r'(http|https)://[^\s]*', 'httpaddr', contents)
    contents = re.sub(r'[^\s]+@[^\s]+','emailaddr', contents)
    contents = re.sub(r'[$]+','dollar', contents)

    words = word_tokenize(contents)

    for i in range(len(words)):
        words[i] = re.sub(r'[^a-zA-Z0-9]','', words[i])
        words[i] = ps.stem(words[i])

    words = [word for word in words if len(word) >= 1]

    return words

In [9]:
x = {'a':2, 'b': 5, 'c': 1}

In [10]:
x.items()

dict_items([('a', 2), ('b', 5), ('c', 1)])

In [11]:
sorted(x.items(), key=lambda z: z[1], reverse=True)

[('b', 5), ('a', 2), ('c', 1)]

In [12]:
def getVocabulary(emails, vocab_length):
    vocabulary=dict()

    for i in range(len(emails)):
        emails[i] = processEmail(emails[i])
        for word in emails[i]:
            if word in vocabulary.keys():
                vocabulary[word] += 1
            else:
                vocabulary[word] = 1

    vocabulary = sorted(vocabulary.items(), key=lambda x: x[1], reverse=True)
    vocabulary = list(map(lambda x: x[0], vocabulary[0:vocab_length]))
    vocabulary = {index: word for index, word in enumerate(vocabulary)}

    return vocabulary

In [17]:
!pip install nltk
import nltk
nltk.download('punkt')

def getVocabulary(emails, vocab_length):
    vocabulary=dict()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [18]:
getVocabulary(data['v2'].to_list(), 2000)

In [19]:
def getKey(dictionary, val):
    for key, value in dictionary.items():
        if value == val:
            return key

In [20]:
def getIndices(email, vocabulary):
    word_indices = set()

    for word in email:
        if word in vocabulary.values():
            word_indices.add(getKey(vocabulary, word))

    return word_indices

In [21]:
email = ('Go until jurong point, crazy Available only in bugis n great world la e buffet Cine there got')

In [22]:
email = processEmail(email)

In [23]:
email

['go',
 'until',
 'jurong',
 'point',
 'crazi',
 'avail',
 'onli',
 'in',
 'bugi',
 'n',
 'great',
 'world',
 'la',
 'e',
 'buffet',
 'cine',
 'there',
 'got']

In [24]:
getIndices(email, vocabulary)

NameError: name 'vocabulary' is not defined

In [26]:
def getIndices(email, vocabulary):
    word_indices = set()

    for word in email:
        if word in vocabulary.values():
            word_indices.add(getKey(vocabulary, word))

    return word_indices

def getKey(my_dict, val):
    for key, value in my_dict.items():
         if val == value:
             return key

# Example vocabulary (replace with your actual vocabulary)
vocabulary = {
    1: 'go',
    2: 'until',
    3: 'jurong',
    # ... add more words and indices
}

email = ('Go until jurong point, crazy Available only in bugis n great world la e buffet Cine there got')
email = processEmail(email)  # Assuming processEmail is defined elsewhere

# Now you can call getIndices
indices = getIndices(email, vocabulary)
print(indices)

{1, 2, 3}


In [27]:
getIndices(email, vocabulary)

{1, 2, 3}

In [28]:
def getFeatureVector(word_indices, vocab_length):
    feature_vec = np.zeros(vocab_length)

    for i in word_indices:
        feature_vec[i] = 1

    return feature_vec

In [29]:
vocab_length = 2000

In [30]:
vocabulary = getVocabulary(data['v2'].to_list(), vocab_length)
emails = data['v2'].to_list()
emails = list(map(lambda x: processEmail(x), emails))

In [32]:
def getIndices(email, vocabulary):
    word_indices = set()
    if vocabulary is not None:  # Check if vocabulary is not None
        for word in email:
            if word in vocabulary.values():
                word_indices.add(getKey(vocabulary, word))
    return word_indices

In [33]:
X = list(map(lambda x :getFeatureVector(getIndices(x, vocabulary), vocab_length), emails))
X = pd.DataFrame(np.array(X).astype(np.int16))

In [34]:
 X

,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5568,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5569,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5570,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
y= data['v1']

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

In [37]:
np.sum(y)/len(y)

0.13406317300789664

In [38]:
model = SVC()
model.fit(X_train, y_train)

SVC()

In [39]:
model.score(X_test, y_test)

0.8582959641255605

In [40]:
y_pred = model.predict(X_test)

In [41]:
f1_score(y_test, y_pred)

0.0